In [1]:
import os, re
import numpy as np
import matplotlib.pyplot as plt

In [2]:
raw_path = "./data/raw/foot/foot_256x256x256_uint8.raw"
shape = (256, 256, 256)    # (Z, Y, X) = (slices, height, width)
dtype = np.uint8

# dataload
vol = np.memmap(raw_path, dtype=dtype, mode="r", shape=shape)

In [3]:
category  = 'foot'
save_root = os.path.join(os.getcwd(), 'data_evaluate', category)

In [4]:
voxel_spacing = (1, 1, 1)  # (z, y, x) mm，可用于3D可视化时设置比例

In [5]:
# 简单校验：文件大小是否匹配
expected_bytes = np.prod(shape) * np.dtype(dtype).itemsize
actual_bytes = os.path.getsize(raw_path)
print(f"Expected bytes: {expected_bytes:,}, Actual bytes: {actual_bytes:,}")

Expected bytes: 16,777,216, Actual bytes: 16,777,216


In [6]:
print("Path       :", raw_path)
print("Shape      :", vol.shape, "(Z, Y, X)")
print("Dtype      :", vol.dtype)
print("Itemsize   :", vol.dtype.itemsize, "bytes")
print("NDIM       :", vol.ndim)

Path       : ./foot/foot_256x256x256_uint8.raw
Shape      : (256, 256, 256) (Z, Y, X)
Dtype      : uint8
Itemsize   : 1 bytes
NDIM       : 3


In [7]:
vmin, vmax = int(vol.min()), int(vol.max())
p1, p50, p99 = np.percentile(vol, [1, 50, 99])
print("\n——grayscale intensity——")
print(f"Min/Max    : {vmin} / {vmax}")
print(f"P1/Median/P99: {p1:.1f} / {p50:.1f} / {p99:.1f}")


——grayscale intensity——
Min/Max    : 0 / 255
P1/Median/P99: 0.0 / 0.0 / 149.0


In [8]:
# Normalize data
def normalize_volume_minmax(volume):
    """最小-最大归一化到 [0, 1] 范围"""
    vol_min = np.min(volume)
    vol_max = np.max(volume)
    print(f"raw data range: [{vol_min}, {vol_max}]")
    
    if vol_min != 0 or vol_max != 1:
        print("Normalize range to [0, 1]")
        volume_normalized = (volume.astype(np.float32) - vol_min) / (vol_max - vol_min + 1e-8)
    else:
        volume_normalized = volume.astype(np.float32)
    
    print(f"normalized range: [{volume_normalized.min():.6f}, {volume_normalized.max():.6f}]")
    return volume_normalized

vol_normalized = normalize_volume_minmax(vol)

vmin, vmax = int(vol_normalized.min()), int(vol_normalized.max())
p1, p50, p995 = np.percentile(vol_normalized, [1, 50, 99.5])
print("\n——grayscale intensity——")
print(f"Min/Max    : {vmin} / {vmax}")
print(f"P1/Median/P99.5: {p1:.1f} / {p50:.1f} / {p995:.1f}")

raw data range: [0, 255]
Normalize range to [0, 1]
normalized range: [0.000000, 1.000000]

——grayscale intensity——
Min/Max    : 0 / 0
P1/Median/P99.5: 0.0 / 0.0 / 0.8


In [9]:
# 灰度累积投影

import os
import time
import numpy as np
from scipy.ndimage import rotate
from tqdm import tqdm
from matplotlib import pyplot as plt

def _pad_to_safe_cube(vol, cval=0.0):
    """
    将(Z,Y,X)体数据填充为边长L的立方体，以避免任意绕Y/Z旋转时被裁剪。
    取 L = ceil(max(Z,Y,X) * sqrt(3))，并将原体素居中放置。
    返回：padded_vol (L,L,L)
    """
    z, y, x = vol.shape
    L = int(np.ceil(max(z, y, x) * np.sqrt(3)))  # 保守上界
    pad_z_total = max(0, L - z)
    pad_y_total = max(0, L - y)
    pad_x_total = max(0, L - x)
    pad_width = (
        (pad_z_total // 2, pad_z_total - pad_z_total // 2),
        (pad_y_total // 2, pad_y_total - pad_y_total // 2),
        (pad_x_total // 2, pad_x_total - pad_x_total // 2),
    )
    padded = np.pad(vol, pad_width, mode="constant", constant_values=cval)
    return padded

def project_volume_orthographic(
    vol_normalized,
    azimuth_deg=0.0,        # 水平转角（绕 Z 轴），x->y 为正
    elevation_deg=30.0,     # 俯仰角（绕 Y 轴），x->z 为正
    spacing=(1.0, 1.0, 1.0),# 体素尺寸(dz, dy, dx)
    mode="sum",             # {"sum","mean","mip"}
    order=1,                # 插值阶数：0=邻近，1=线性，3=三次样条
    cval=0.0,               # 旋转边界填充值
    safe_same_size=True     # True: 先pad到安全立方体，并固定输出尺寸
):
    """
    返回 2D 投影(np.float32)。旋转顺序：先绕Y轴(elevation)，再绕Z轴(azimuth)；投影方向为旋转后体数据的+Z方向。
    """
    vol = vol_normalized.astype(np.float32, copy=False)
    if safe_same_size:
        vol = _pad_to_safe_cube(vol, cval=cval)

    dz, dy, dx = spacing


    
    # 1) 俯仰角（绕 X 轴，上下倾斜）保持不变
    vol_rot = rotate(vol, angle=elevation_deg, axes=(1, 0), reshape=False,
                 order=order, mode='constant', cval=cval)

    # 2) 左右扫动（绕 Y 轴）
    vol_rot = rotate(vol_rot, angle=azimuth_deg, axes=(0, 2), reshape=False,
                 order=order, mode='constant', cval=cval)


    # 3) 沿 Z 轴做投影
    if mode == "sum":
        proj = vol_rot.sum(axis=0) * dz
    elif mode == "mean":
        proj = vol_rot.mean(axis=0)
    elif mode == "mip":
        proj = vol_rot.max(axis=0)
    else:
        raise ValueError("mode must be one of {'sum','mean','mip'}")

    return proj.astype(np.float32, copy=False)

def save_rotation_projections(
    vol_normalized,
    save_root,
    elevation_deg=0.0,
    mode="sum",
    step_deg=3.0,           # 每帧角度增量
    total_deg=360.0,        # 总角度
    order=1,
    cval=0.0,
    spacing=(1.0,1.0,1.0),
    progress_desc="Saving projections"
):
    """
    绕 Y 轴生成投影，输出目录和文件名格式：
    - 目录: Proj_elevation_{elevation_deg}
    - 文件: angle_{角度}.png
    """
    assert step_deg > 0, "step_deg 必须为正数"
    n_frames = int(np.ceil(total_deg / step_deg))
    azimuths = (np.arange(n_frames) * step_deg) % 360.0

    # 输出目录
    out_dir = os.path.join(save_root, f"Proj_elevation_{elevation_deg}")
    os.makedirs(out_dir, exist_ok=True)

    t0 = time.time()
    saved = []

    with tqdm(total=n_frames, desc=progress_desc) as pbar:
        for i, az in enumerate(azimuths):
            proj = project_volume_orthographic(
                vol_normalized,
                azimuth_deg=float(az),
                elevation_deg=float(elevation_deg),
                spacing=spacing,
                mode=mode,
                order=order,
                cval=cval,
                safe_same_size=True
            )
            # 文件名改为 angle_x.png
            fname = os.path.join(out_dir, f"angle_{int(round(az))}.png")
            plt.imsave(fname, proj, cmap="gray")
            saved.append((float(az), fname))
            pbar.set_postfix({"angle_deg": f"{az:.1f}"})
            pbar.update(1)

    elapsed = time.time() - t0
    print(f"⏱️ Total time: {elapsed:.2f} s | Frames: {n_frames} | Output dir: {out_dir}")

    return {
        "out_dir": out_dir,
        "frames": saved,
        "elapsed_sec": elapsed,
        "n_frames": n_frames
    }


In [10]:
import os
import numpy as np
from matplotlib import pyplot as plt
from skimage.metrics import structural_similarity as ssim  # pip install scikit-image

def _metrics(a, b):
    diff = a - b
    mae = float(np.mean(np.abs(diff)))
    mse = float(np.mean(diff**2))
    pmin = float(min(a.min(), b.min()))
    pmax = float(max(a.max(), b.max()))
    data_range = max(1e-8, pmax - pmin)
    ssim_val = float(ssim(a, b, data_range=data_range))
    return mae, mse, ssim_val, diff, data_range

def compare_central_symmetric_projections(
    vol_normalized,
    save_root,
    elevation_deg=0.0,
    thetas_deg=(0, 10, 20, 30, 45, 60, 90),  # 基准角 θ 列表；自动配对 θ 与 θ+180°
    mode="sum",
    order=1,
    cval=0.0,
    spacing=(1.0, 1.0, 1.0),
    cmap="gray",
):
    """
    比较 θ 与 (θ+180°) 的中心对称关系，并同时评估：
      - raw:        proj(θ) vs proj(θ+180)
      - lr:         proj(θ) vs fliplr(proj(θ+180))
      - rot180:     proj(θ) vs rot90(proj(θ+180), 2)

    输出目录:
      save_root/Proj_elevation_{elevation_deg}_centersym/pair_theta_{θ}
    保存:
      angle_{θ}.png, angle_{θp}.png, diff_abs_{raw/lr/rot180}.png, diff_report.txt
    返回:
      包含每个 θ 的三种对齐方式的 MAE/MSE/SSIM 的汇总。
    """
    base_dir = os.path.join(save_root, f"Proj_elevation_{elevation_deg}_centersym")
    os.makedirs(base_dir, exist_ok=True)
    results = []

    for theta in thetas_deg:
        theta = float(theta)
        theta_p = (theta + 180.0) % 360.0

        # 生成两幅投影
        proj_a = project_volume_orthographic(
            vol_normalized, azimuth_deg=theta, elevation_deg=elevation_deg,
            spacing=spacing, mode=mode, order=order, cval=cval, safe_same_size=True
        )
        proj_b = project_volume_orthographic(
            vol_normalized, azimuth_deg=theta_p, elevation_deg=elevation_deg,
            spacing=spacing, mode=mode, order=order, cval=cval, safe_same_size=True
        )

        # 三种对齐关系的指标
        mae_raw, mse_raw, ssim_raw, diff_raw, dr_raw = _metrics(proj_a, proj_b)
        mae_lr, mse_lr, ssim_lr, diff_lr, dr_lr = _metrics(proj_a, np.fliplr(proj_b))

        pair_dir = os.path.join(base_dir, f"pair_theta_{int(round(theta))}")
        os.makedirs(pair_dir, exist_ok=True)

        # 保存原投影
        f_a = os.path.join(pair_dir, f"angle_{int(round(theta))}.png")
        f_b = os.path.join(pair_dir, f"angle_{int(round(theta_p))}.png")
        plt.imsave(f_a, proj_a, cmap=cmap)
        plt.imsave(f_b, proj_b, cmap=cmap)

        # 保存差异热力图（归一化可视化）
        def _save_abs_diff(fname, D):
            Dabs = np.abs(D)
            plt.imsave(fname, Dabs / (Dabs.max() + 1e-8), cmap="inferno")

        _save_abs_diff(os.path.join(pair_dir, "diff_abs_raw.png"), diff_raw)
        _save_abs_diff(os.path.join(pair_dir, "diff_abs_lr.png"), diff_lr)


        # 文本报告
        f_txt = os.path.join(pair_dir, "diff_report.txt")
        with open(f_txt, "w", encoding="utf-8") as f:
            f.write(
                f"elevation_deg={elevation_deg}\n"
                f"theta={theta} deg, theta_p=(theta+180)={theta_p} deg\n"
                f"mode={mode}, order={order}, spacing={spacing}\n"
                f"min/max(A)={proj_a.min():.6g}/{proj_a.max():.6g}\n"
                f"min/max(B)={proj_b.min():.6g}/{proj_b.max():.6g}\n"
                "\n-- RAW (A vs B) --\n"
                f"MAE={mae_raw:.6g} | MSE={mse_raw:.6g} | SSIM={ssim_raw:.6g} | data_range={dr_raw:.6g}\n"
                "-- LR  (A vs fliplr(B)) --\n"
                f"MAE={mae_lr:.6g} | MSE={mse_lr:.6g} | SSIM={ssim_lr:.6g} | data_range={dr_lr:.6g}\n"
                "-- ROT180 (A vs rot90(B,2)) --\n"
                f"A_path={f_a}\nB_path={f_b}\n"
            )

        print(
            f"[θ={theta:.1f}°, θp={theta_p:.1f}°] "
            f"RAW  -> MAE={mae_raw:.4g}, MSE={mse_raw:.4g}, SSIM={ssim_raw:.4f} | "
            f"LR   -> MAE={mae_lr:.4g}, MSE={mse_lr:.4g}, SSIM={ssim_lr:.4f} | "
        )

        results.append({
            "theta_deg": theta,
            "theta_p_deg": theta_p,
            "raw":  {"mae": mae_raw,  "mse": mse_raw,  "ssim": ssim_raw},
            "lr":   {"mae": mae_lr,   "mse": mse_lr,   "ssim": ssim_lr},
            "paths": {"A": f_a, "B": f_b, "report": f_txt}
        })

    return {
        "base_dir": base_dir,
        "elevation_deg": elevation_deg,
        "mode": mode,
        "results": results
    }


In [12]:
out = compare_central_symmetric_projections(
        vol_normalized=vol_normalized,
        save_root=save_root,
        elevation_deg=30.0,          
        thetas_deg=(10, 20, 30, 45), 
        mode="sum",
        order=1,
        cval=0.0,
        spacing=(1.0, 1.0, 1.0),
    )



[θ=10.0°, θp=190.0°] RAW  -> MAE=3.606, MSE=72.77, SSIM=0.7353 | LR   -> MAE=9.17e-07, MSE=9.437e-12, SSIM=1.0000 | 
[θ=20.0°, θp=200.0°] RAW  -> MAE=3.266, MSE=54.49, SSIM=0.7269 | LR   -> MAE=8.775e-07, MSE=7.626e-12, SSIM=1.0000 | 
[θ=30.0°, θp=210.0°] RAW  -> MAE=2.918, MSE=43.23, SSIM=0.7286 | LR   -> MAE=8.558e-07, MSE=6.51e-12, SSIM=1.0000 | 
[θ=45.0°, θp=225.0°] RAW  -> MAE=2.7, MSE=36.37, SSIM=0.6996 | LR   -> MAE=8.329e-07, MSE=5.676e-12, SSIM=1.0000 | 
